In [1]:
import os
import pandas as pd
import numpy as np
import json

In [ ]:
sf_police_budget_data = pd.read_csv('./sanfranciscodataset/plot_data/total_police_spending.csv')

In [ ]:
sf_police_budget_data = sf_police_budget_data.to_json(orient='records')[1:-1].replace('},{', '} {')

In [ ]:

with open('sf_police_budget_data.json', 'w') as f:
    json.dump(sf_police_budget_data, f)

In [ ]:
#SF incidents data
incidents = pd.read_csv('./sanfranciscodataset/plot_data/yearly_reported_incidents.csv')

In [ ]:
sf_incidents_json = incidents.to_json(orient='records')[1:-1].replace('},{', '} {')

In [ ]:
with open('sf_incidents.json', 'w') as f:
    json.dump(sf_incidents_json, f)

In [ ]:
#SF UOF
sf_force = pd.read_csv('./sanfranciscodataset/plot_data/sf_police_uof.csv')

In [ ]:
sf_force_json = sf_force.to_json(orient='records')[1:-1].replace('},{', '} {').replace('} {', '}, {')

In [ ]:
#Spending data
df = pd.read_csv('Spending_Revenue_cleaned.csv', sep=',')

df_police = df[df['Department'] == 'POL Police']
years = df['Fiscal Year'].unique()
years
df_year = []
for index, year in enumerate(df_police['Fiscal Year'].unique()):
    df_year.append( df[df['Fiscal Year'] == year])
    
program_sums = []
for i in range(19):
    program_sums.append( pd.DataFrame(df_year[i].groupby('Program').Amount.agg('sum')) )
    col_year = years[i]
    program_sums[i].columns = [col_year]

important_programs = ['Patrol', 'Operations And Administration', 'Special Operations',
                      'SFPD-Recruitment And Examination Program', 'SFPD Training']
df_agg_programs=pd.concat(program_sums, axis=1)
df_agg_programs.fillna(0, inplace=True)
df_agg_programs = df_agg_programs.loc[important_programs]
df_agg_programs
df_agg_programs = df_agg_programs.T
df_agg_programs = df_agg_programs.reset_index()
df_agg_programs.columns = ['Year', 'Patrol', 'Operations And Administration', 'Special Operations',
                      'SFPD-Recruitment And Examination Program', 'SFPD Training']
df_agg_programs

In [3]:
sf_police_budget_detail = df_agg_programs.to_json(orient='records')[1:-1].replace('},{', '} {').replace('} {', '}, {')
sf_police_budget_detail

'{"Year":1999,"Patrol":156665669.1800000072,"Operations And Administration":41102684.790000014,"Special Operations":151017.92,"SFPD-Recruitment And Examination Program":1292994.8900000004,"SFPD Training":1490869.8500000001}, {"Year":2000,"Patrol":176313685.9399999976,"Operations And Administration":53292740.0099999681,"Special Operations":305729.53,"SFPD-Recruitment And Examination Program":1544261.3999999999,"SFPD Training":1380405.97}, {"Year":2001,"Patrol":186723778.8900001049,"Operations And Administration":58172934.3600000069,"Special Operations":163570.07,"SFPD-Recruitment And Examination Program":1735868.5200000003,"SFPD Training":3000961.2400000002}, {"Year":2002,"Patrol":194740096.1700000465,"Operations And Administration":52728335.9700000063,"Special Operations":291461.02,"SFPD-Recruitment And Examination Program":1344727.5800000001,"SFPD Training":2465499.8900000001}, {"Year":2003,"Patrol":190460560.4300001264,"Operations And Administration":49064986.159999989,"Special Opera

In [4]:
with open('sf_police_budget_detail.json', 'w') as f:
    json.dump(sf_police_budget_detail, f)

In [27]:
#Population Share
pop_2010 = pd.read_csv('Intercensal_2000-2010_Total_Age-Race.xls - Total.csv', parse_dates=True)
pop_2020 = pd.read_csv('P3-Excel-Orange-YubaCos.xlsx - Orange to Yuba.csv')

In [28]:
pop_2010_long = pop_2010.melt(id_vars=['CountyCode', 'CountyName', 'RaceCode', 'RaceName', 'Gender', 'Age'], var_name='year', value_name='population')
pop_2010_long = pop_2010_long.loc[(pop_2010_long['Gender'] == 'Total') & (pop_2010_long['CountyCode'] <= 58), ['CountyName', 'RaceName', 'year', 'population']]
pop_2010_long

,CountyName,RaceName,year,population
0,Alameda,White,4/1/2000,5575
1,Alameda,White,4/1/2000,5550
2,Alameda,White,4/1/2000,5573
3,Alameda,White,4/1/2000,5632
4,Alameda,White,4/1/2000,5884
...,...,...,...,...
499844,Yuba,Multi-Race,4/1/2010,0
499845,Yuba,Multi-Race,4/1/2010,0
499846,Yuba,Multi-Race,4/1/2010,1
499847,Yuba,Multi-Race,4/1/2010,0


In [29]:
pop_2010_long['year'] = pd.to_datetime(pop_2010_long.year)
pop_2010_long.year = pop_2010_long.year.dt.year
pop_2010_long = pop_2010_long.groupby(['CountyName', 'RaceName', 'year']).population.sum().reset_index()
pop_2010_long

,CountyName,RaceName,year,population
0,Alameda,American Indian,2000,10808
1,Alameda,American Indian,2001,5145
2,Alameda,American Indian,2002,4937
3,Alameda,American Indian,2003,4774
4,Alameda,American Indian,2004,4633
...,...,...,...,...
4461,Yuba,White,2006,42714
4462,Yuba,White,2007,42991
4463,Yuba,White,2008,43201
4464,Yuba,White,2009,42833


In [30]:
pop_2010_long.RaceName.unique()

array(['American Indian', 'Asian', 'Black', 'Hispanic', 'Multi-Race',
       'Pacific Islander', 'White'], dtype=object)

In [58]:
pop_2020 = pop_2020[['County Name', 'Race Name', 'year', 'Total Pop']]
pop_2020.columns = ['CountyName', 'RaceName', 'year', 'population']
pop_2020.CountyName = pop_2020.CountyName.str.replace(' County', '')

KeyError: "['Total Pop', 'Race Name', 'County Name'] not in index"

In [59]:
pop_2020 = pop_2020.groupby(['CountyName', 'RaceName', 'year']).population.sum().reset_index()
pop_2020

,CountyName,RaceName,year,population
0,Orange,American Indian,2010,6316
1,Orange,American Indian,2011,6502
2,Orange,American Indian,2012,6684
3,Orange,American Indian,2013,6871
4,Orange,American Indian,2014,7151
...,...,...,...,...
10348,Yuba,White,2056,47069
10349,Yuba,White,2057,47121
10350,Yuba,White,2058,47296
10351,Yuba,White,2059,47454


In [60]:
pop_2020.RaceName = pop_2020.RaceName.str.replace(', Non-Hispanic', '')
pop_2020.RaceName = pop_2020.RaceName.str.replace('American Indian or Alaska Native', 'American Indian')
pop_2020.RaceName = pop_2020.RaceName.str.replace('Hispanic \(any race\)', 'Hispanic')
pop_2020.RaceName = pop_2020.RaceName.str.replace('Native Hawaiian or Pacific Islander', 'Pacific Islander')
pop_2020.RaceName = pop_2020.RaceName.str.replace('Multiracial \(two or more of above races\)', 'Multi-Race')
pop_2020.RaceName.unique()

<ipython-input-60-b598783ce459>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  pop_2020.RaceName = pop_2020.RaceName.str.replace('Hispanic \(any race\)', 'Hispanic')
<ipython-input-60-b598783ce459>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  pop_2020.RaceName = pop_2020.RaceName.str.replace('Multiracial \(two or more of above races\)', 'Multi-Race')


array(['American Indian', 'Asian', 'Black', 'Hispanic', 'Multi-Race',
       'Pacific Islander', 'White'], dtype=object)

In [61]:
pop_data_all_years = pop_2010_long.append(pop_2020)

In [62]:
pop_data_all_years.RaceName.unique()

array(['American Indian', 'Asian', 'Black', 'Hispanic', 'Multi-Race',
       'Pacific Islander', 'White'], dtype=object)

In [63]:
sf_pop_data = pop_data_all_years.loc[(pop_data_all_years['CountyName'] == 'San Francisco') & (pop_data_all_years['year'] <= 2021)]

In [64]:
#sf_pop_data.to_csv('sf_pop_data_by_race.csv')

In [65]:
sf_pop_data['total_pop'] = sf_pop_data.groupby(['CountyName', 'year']).population.transform('sum')
sf_pop_data['pop_share'] = (sf_pop_data['population'] / sf_pop_data['total_pop']) * 100


<ipython-input-65-a5527a49e1c3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sf_pop_data['total_pop'] = sf_pop_data.groupby(['CountyName', 'year']).population.transform('sum')
<ipython-input-65-a5527a49e1c3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sf_pop_data['pop_share'] = (sf_pop_data['population'] / sf_pop_data['total_pop']) * 100


In [66]:
sf_pop_data[sf_pop_data['RaceName'] == 'Asian']

,CountyName,RaceName,year,population,total_pop,pop_share
2907,San Francisco,Asian,2010,269561,810369,33.263982
2908,San Francisco,Asian,2011,275624,821235,33.562135
2909,San Francisco,Asian,2012,281431,834572,33.721596
2910,San Francisco,Asian,2013,287401,845063,34.009417
2911,San Francisco,Asian,2014,294461,854587,34.456527
2912,San Francisco,Asian,2015,301673,864405,34.899497
2913,San Francisco,Asian,2016,307470,872281,35.248962
2914,San Francisco,Asian,2017,313760,880343,35.640654
2915,San Francisco,Asian,2018,318959,886618,35.974794
2916,San Francisco,Asian,2019,322798,889360,36.295538


In [67]:
uof = pd.read_csv('subject_uof_race_by_year.csv')

In [68]:
uof

,Unnamed: 0,year,race,count
0,0,2015,American Indian,1
1,1,2015,Asian or Pacific Islander,6
2,2,2015,Black,27
3,3,2015,Hispanic,17
4,4,2015,Unknown,1
5,5,2015,White,24
6,6,2015,NaN,699
7,7,2016,American Indian,1
8,8,2016,Asian or Pacific Islander,139
9,9,2016,Black,786


In [69]:
uof = uof.drop('Unnamed: 0', axis=1)

In [70]:
uof['race'].unique()

array(['American Indian', 'Asian or Pacific Islander', 'Black',
       'Hispanic', 'Unknown', 'White', nan], dtype=object)

In [71]:
sf_pop_data.RaceName.unique()
sf_pop_data.loc[sf_pop_data['RaceName'] == 'Asian', 'RaceName'] = 'Asian or Pacific Islander'
sf_pop_data.loc[sf_pop_data['RaceName'] == 'Pacific Islander', 'RaceName'] = 'Asian or Pacific Islander'

sf_pop_data_clean = sf_pop_data.groupby(['CountyName', 'RaceName', 'year']).population.sum().reset_index()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [72]:
sf_pop_data_clean[sf_pop_data_clean['RaceName'] == 'Asian or Pacific Islander']

,CountyName,RaceName,year,population
12,San Francisco,Asian or Pacific Islander,2010,272675
13,San Francisco,Asian or Pacific Islander,2011,278577
14,San Francisco,Asian or Pacific Islander,2012,284159
15,San Francisco,Asian or Pacific Islander,2013,290014
16,San Francisco,Asian or Pacific Islander,2014,297058
17,San Francisco,Asian or Pacific Islander,2015,304303
18,San Francisco,Asian or Pacific Islander,2016,310033
19,San Francisco,Asian or Pacific Islander,2017,316256
20,San Francisco,Asian or Pacific Islander,2018,321354
21,San Francisco,Asian or Pacific Islander,2019,325006


In [73]:
sf_pop_data_clean.RaceName.unique()

array(['American Indian', 'Asian or Pacific Islander', 'Black',
       'Hispanic', 'Multi-Race', 'White'], dtype=object)

In [74]:
sf_uof_by_race = sf_pop_data_clean.merge(uof, left_on=['year', 'RaceName'], right_on=['year', 'race'])
sf_uof_by_race

,CountyName,RaceName,year,population,race,count
0,San Francisco,American Indian,2015,2718,American Indian,1
1,San Francisco,American Indian,2016,2879,American Indian,1
2,San Francisco,American Indian,2019,3479,American Indian,2
3,San Francisco,American Indian,2020,3560,American Indian,1
4,San Francisco,Asian or Pacific Islander,2015,304303,Asian or Pacific Islander,6
5,San Francisco,Asian or Pacific Islander,2016,310033,Asian or Pacific Islander,139
6,San Francisco,Asian or Pacific Islander,2017,316256,Asian or Pacific Islander,87
7,San Francisco,Asian or Pacific Islander,2018,321354,Asian or Pacific Islander,84
8,San Francisco,Asian or Pacific Islander,2019,325006,Asian or Pacific Islander,53
9,San Francisco,Asian or Pacific Islander,2020,328574,Asian or Pacific Islander,15


In [76]:
sf_uof_by_race['rate_per_100k'] = (sf_uof_by_race['count'] / sf_uof_by_race['population']) * 100000
sf_uof_by_race.rate_per_100k = sf_uof_by_race.rate_per_100k.round()

In [77]:
sf_uof_by_race

,CountyName,RaceName,year,population,race,count,rate_per_100k
0,San Francisco,American Indian,2015,2718,American Indian,1,37.0
1,San Francisco,American Indian,2016,2879,American Indian,1,35.0
2,San Francisco,American Indian,2019,3479,American Indian,2,57.0
3,San Francisco,American Indian,2020,3560,American Indian,1,28.0
4,San Francisco,Asian or Pacific Islander,2015,304303,Asian or Pacific Islander,6,2.0
5,San Francisco,Asian or Pacific Islander,2016,310033,Asian or Pacific Islander,139,45.0
6,San Francisco,Asian or Pacific Islander,2017,316256,Asian or Pacific Islander,87,28.0
7,San Francisco,Asian or Pacific Islander,2018,321354,Asian or Pacific Islander,84,26.0
8,San Francisco,Asian or Pacific Islander,2019,325006,Asian or Pacific Islander,53,16.0
9,San Francisco,Asian or Pacific Islander,2020,328574,Asian or Pacific Islander,15,5.0


In [78]:
sf_uof_by_race = sf_uof_by_race[['year', 'race', 'rate_per_100k']].pivot(index='year', columns=['race'], values='rate_per_100k').reset_index()
sf_uof_by_race

race,year,American Indian,Asian or Pacific Islander,Black,Hispanic,White
0,2015,37.0,2.0,56.0,13.0,7.0
1,2016,35.0,45.0,1627.0,300.0,113.0
2,2017,NaN,28.0,1387.0,276.0,108.0
3,2018,NaN,26.0,1274.0,224.0,113.0
4,2019,57.0,16.0,978.0,188.0,79.0
5,2020,28.0,5.0,304.0,78.0,29.0


In [79]:
sf_uof_by_race.columns

Index(['year', 'American Indian', 'Asian or Pacific Islander', 'Black',
       'Hispanic', 'White'],
      dtype='object', name='race')

In [80]:
sf_uof_by_race = sf_uof_by_race[sf_uof_by_race['year'] > 2015]

In [81]:
sf_uof_by_race_json = sf_uof_by_race.to_json(orient='records')[1:-1].replace('},{', '} {').replace('} {', '}, {')
sf_uof_by_race_json

'{"year":2016,"American Indian":35.0,"Asian or Pacific Islander":45.0,"Black":1627.0,"Hispanic":300.0,"White":113.0}, {"year":2017,"American Indian":null,"Asian or Pacific Islander":28.0,"Black":1387.0,"Hispanic":276.0,"White":108.0}, {"year":2018,"American Indian":null,"Asian or Pacific Islander":26.0,"Black":1274.0,"Hispanic":224.0,"White":113.0}, {"year":2019,"American Indian":57.0,"Asian or Pacific Islander":16.0,"Black":978.0,"Hispanic":188.0,"White":79.0}, {"year":2020,"American Indian":28.0,"Asian or Pacific Islander":5.0,"Black":304.0,"Hispanic":78.0,"White":29.0}'

In [82]:
with open('sf_race_uof_and_pop_share.json', 'w') as f:
    json.dump(sf_uof_by_race_json, f)